In [1]:
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import os
import torch.optim as optim
from probe_experiment import ProbeExperiment
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from io_utils import *
from math import ceil

In [2]:
annotations = read_pkl('data/filtered_annotations.pkl')
annotations.sort(key = lambda ant : ant['f_name'])

from annotated_datasets import *
train_ants = annotations[:int(len(annotations)*0.9)]
test_ants = annotations[int(len(annotations)*0.9):]
train_dataset = LengthDataset(train_ants)
test_dataset = LengthDataset(test_ants)
train_dataset = load_to_memory(train_dataset)
test_dataset = load_to_memory(test_dataset)

from torch.utils.data import DataLoader
def get_train_loader():
    return DataLoader(train_dataset, shuffle=True, batch_size=512)

def get_test_loader():
    return DataLoader(test_dataset, shuffle=False, batch_size=512)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23407/23407 [00:50<00:00, 461.28it/s]


In [4]:
from models import *
go_model = load_go_model_from_ckpt('model_ckpt.pth.tar', rm_prefix=True)
feat_models = [CutModel(go_model, i).cuda() for i in range(8)]
n_channels = [8, 64, 64, 64, 48, 48, 32, 32]
probe_models = [nn.Sequential(nn.Flatten(), nn.Linear(nc*19*19, 1)).cuda() for nc in n_channels]

In [ ]:
def unsqueezeBCE(pred, target):
    return nn.BCEWithLogitsLoss()(pred, target.unsqueeze(dim=1))
criterion = unsqueezeBCE

aucs = []
for depth in range(0, 8):
    print('probing at depth %d' % depth)
    feat_model = feat_models[depth]
    probe_model = probe_models[depth]
    exp = ProbeExperiment(get_train_loader, get_test_loader, feat_model)
    config = {'name':'len/len_d%d'%depth, 'write_log':True, 'progress_bar':True, 'save_ckpt':False}
    optimizer = torch.optim.Adam(probe_model.parameters())
    exp.run(probe_model, criterion, optimizer, 5, config)
    preds, labels = exp.get_predictions(probe_model, get_test_loader())
    auc = roc_auc_score(labels.flatten(), preds.flatten())
    aucs.append(auc)
    print(auc)

  0%|▎                                                                                                                           | 1/412 [00:00<01:05,  6.25it/s]

probing at depth 0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 412/412 [00:30<00:00, 13.51it/s]


[LOG] epoch 0 loss 0.680337, new best


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 412/412 [00:28<00:00, 14.53it/s]


...

  0%|▌                                                                                                                           | 2/412 [00:00<00:33, 12.12it/s]

[LOG] epoch 2 loss 0.690547


  0%|▌                                                                                                                           | 2/412 [00:00<00:34, 11.97it/s]

[LOG] epoch 3 loss 0.694746


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 412/412 [00:36<00:00, 11.23it/s]
